In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from xopt.vocs import VOCS, ObjectiveEnum

In [34]:
Y = """
variables:
  a: [0, 1e6]
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234
  h: ['any', [{'thi':'ng'}]]

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234, 'h': ['any', [{'thi': 'ng'}]]}, linked_variables=None)

In [41]:
# Make dict and re-parse
vocs2 = VOCS.parse_obj(vocs.dict())

In [40]:
# Check that these are the same
vocs2 == vocs


True

In [42]:
vocs.json()

'{"variables": {"a": [0.0, 1000000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234, "h": ["any", [{"thi": "ng"}]]}, "linked_variables": null}'

True

In [52]:

import json
import numpy as np
class NpEncoder(json.JSONEncoder):
    """
    See: https://stackoverflow.com/questions/50916422/python-typeerror-object-of-type-int64-is-not-json-serializable/50916741
    """

    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif hasattr(obj, "dict"): # Works with PyDantic models
            return obj.dict()            
        else:
            return super(NpEncoder, self).default(obj)

d = {'a':vocs}
json.dumps(d, cls=NpEncoder)

'{"a": {"variables": {"a": [0.0, 1000000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234, "h": ["any", [{"thi": "ng"}]]}, "linked_variables": null}}'

In [27]:
# Limits of variable 'a'
vocs.variables['a']

[0.0, 1000000.0]

In [28]:
# Limits of objective 'c'
vocs.objectives['c'] 

'MAXIMIZE'

In [30]:
vocs.objectives['c']  == ObjectiveEnum.MAXIMIZE

True